# UI2
updates
1. pick labels
2. change prob acc

## Pick the labels that you want to use here

In [1]:
selected_labels = ["door_nock","glass_shatter","car_horn","dog_bark","drilling","nothing","siren","nothing2"]

## sound_rate (only raspberrypi use)

In [2]:
# text = '''pcm.!default {
#     type asym
#     capture.pcm "mic"
#     playback.pcm "speaker"
# }
# pcm.mic {
#     type plug
#     slave {
#         pcm "hw:2,0"
#     }
# }
# pcm.speaker {
#     type plug
#     slave {
#         pcm "hw:1,0"
#     }
# }'''

# # 파일 경로와 이름 설정
# file_path = "/home/pi/.asoundrc"

# # 텍스트 파일 생성 및 저장
# with open(file_path, "w") as file:
#     file.write(text)

# print("Text saved to:", file_path)

In [3]:
import os
current_path = os.getcwd()
print("현재 경로:", current_path)

현재 경로: /Volumes/새 볼륨/GARAGE/@DUNE/UI


In [4]:
# Relative path to the model file
relative_path_to_model = "../@AI/Sound Classification/ResNet18_02.pth"

# Combine the current path and the relative path to create the absolute path to the model
path_to_model = os.path.join(current_path, relative_path_to_model)

print("Path to the model:", path_to_model)

Path to the model: /Volumes/새 볼륨/GARAGE/@DUNE/UI/../@AI/Sound Classification/ResNet18_02.pth


In [5]:
import torch
import torch.nn as nn
from torchvision.models import resnet18

#One GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet18(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(512, len(selected_labels))
try:
    state_dict = torch.load(path_to_model, map_location=device)
    new_state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}
    model.load_state_dict(new_state_dict)
    model = model.to(device)
    model = model.eval()
    print("Model successfully loaded deivce : ",device)
except:
    print("Failed to load the model. Please check the model file.")
    

/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Failed to load the model. Please check the model file.


In [6]:
import torchaudio
from torchvision import transforms  # Import the transforms module


#Transform

SAMPLE_RATE = 22050

class MonoToColor(nn.Module):
    def __init__(self, num_channels=3):
        super(MonoToColor, self).__init__()
        self.num_channels = num_channels

    def forward(self, tensor):
        return tensor.repeat(self.num_channels, 1, 1)

# Apply the same transformation as used during training
transformation = transforms.Compose([
    torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE, n_mels=128),
    torchaudio.transforms.AmplitudeToDB(stype='power', top_db=80),
    MonoToColor()
])

/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/torchaudio/functional/functional.py:576: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


# Speech to text

### Import lib

In [7]:
import pyaudio
import speech_recognition as sr
import time

### Read audio func

In [8]:
def record_audio():
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=1, rate=16000, frames_per_buffer=1024, input=True)
    frames = []
    for i in range(0, int(16000 / 1024 * 3)):
        data = stream.read(1024)
        frames.append(data)
    stream.close()
    audio.terminate()  # close the PyAudio object
    return b''.join(frames)

### Return text

In [9]:
def transcribe_audio(audio):
    recognizer = sr.Recognizer()
    audio_data = sr.AudioData(audio, 16000, 2)
    try:
        text = recognizer.recognize_sphinx(audio_data)
        
        return text
    except:
        return None
        

### Spinx Main

In [10]:
def main(times):
    for i in range(0, times):
        audio = record_audio()
        text = transcribe_audio(audio)
        print(text)

### RUN

In [11]:
TIMES = 0
if __name__ == "__main__":
    main(TIMES)

# SOUND EFFECT

In [12]:
import time
import torch.nn.functional as F
import sounddevice as sd

print("device : ",device)
## print every labels
def continuous_sound_prediction(model, device, transformation, sample_rate, target_sample_rate):
    # Define class labels

    # Record a 2 seconds mono audio at the specified sample rate
    duration = 2.0  # seconds
    recording = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1) 
    sd.wait()

    # Convert to PyTorch tensor and switch channels and frames
    recording = torch.from_numpy(recording).float()
    recording = torch.transpose(recording, 0, 1)

    # Resample if necessary
    if sample_rate != target_sample_rate:
        resampler = torchaudio.transforms.Resample(sample_rate, target_sample_rate)
        recording = resampler(recording)

    # Mix down if necessary
    if recording.shape[0] > 1:
        recording = torch.mean(recording, dim=0, keepdim=True)

    # Cut or pad if necessary
    if recording.shape[1] > target_sample_rate:
        recording = recording[:, :target_sample_rate]
    elif recording.shape[1] < target_sample_rate:
        num_missing_samples = target_sample_rate - recording.shape[1]
        last_dim_padding = (0, num_missing_samples)
        recording = nn.functional.pad(recording, last_dim_padding)

    # Apply transformation
    recording = transformation(recording)

    # Make the prediction
    model.eval()  # set model to evaluation mode
    with torch.no_grad():  # deactivate autograd engine to reduce memory usage and speed up computations
        recording = recording.to(device)
        outputs = model(recording[None, ...])
        #probabilities = F.softmax(outputs, dim=1)  # apply softmax to output (for 100%)
        #_, predicted = torch.max(outputs, 1)
        probabilities = torch.sigmoid(outputs)  # apply sigmoid to output (for indivisual points)
        _, predicted = torch.max(outputs, 1)

    # Get predicted label and its corresponding probability
    predicted_label = selected_labels[predicted.item()]
    predicted_confidence = probabilities[0, predicted.item()].item()  # get the probability of the predicted class

    ######## Adjust 'x' probability   #########
    change_label = "glass_shatter"
    change_probability = 0.5
    try:# if you have something to reduce
        x_index = selected_labels.index(change_label)
        probabilities[0, x_index] = max(0.0, probabilities[0, x_index].item() - change_probability)
    except:#if you dont
        pass
    # Print the probabilities of all labels in one line
    #prob_strs = [f"{label} {probabilities[0, idx].item():.2%}" for idx, label in enumerate(selected_labels)]
    #print(f"/ ".join(prob_strs))

    if predicted_confidence > 0.0:
        return predicted_label, predicted_confidence, probabilities
    else:
        return "NONE" , 0 , ""


device :  cpu


In [13]:
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtCore import QThread, pyqtSignal
from PyQt5.QtGui import QMovie

from time import sleep
import torch
import torch.nn as nn
import pyaudio
import torchaudio
import sounddevice as sd
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.autograd import Variable


###### INIT ######
p = pyaudio.PyAudio()
sample_rate = SAMPLE_RATE
target_sample_rate = SAMPLE_RATE
mode = 0
print(" 결과 값 : ")





# Sound Analysis class running on a separate thread
class SoundAnalysis(QThread):
    # Define a pyqtSignal with str type, which will be used to send the analysis results to the main thread
    result_signal = pyqtSignal(str, float)  # Add a float type for the probability

    def __init__(self, model, device, transformation, sample_rate):
        QThread.__init__(self)
        self.model = model
        self.device = device
        self.transformation = transformation
        self.sample_rate = sample_rate
        

    def run(self):## Main code
        count = 0
        text_str = ""
        while True:

            if mode == True:
                text_str = "" #empty the text strings

                predicted_label, predicted_confidence, probabilities = continuous_sound_prediction(model, device, transformation, SAMPLE_RATE, SAMPLE_RATE) 
                if mode == True:
                    try:               
                        self.result_signal.emit(predicted_label, predicted_confidence)
                        prob_strs = [f"{label} {probabilities[0, idx].item():.2%}" for idx, label in enumerate(selected_labels)]
                        #vibration()
                        print(f"\r{count} / " + " / ".join(prob_strs), end="")
                    except:
                        self.result_signal.emit(predicted_label, predicted_confidence)
                    count = count + 1
            if mode == False:

                times = 2
                for _ in range(0, times):
                    audio = record_audio()
                    text = transcribe_audio(audio)
                    print(text)
                    text_str += text + " "
                    if mode == True: ## This is checking again if the button is pushed. If I dont add this, it will reset_label and show the text_str again before going to mode True
                        break
                    self.result_signal.emit(text_str,0)
        
    def reset_label(self):# Restting the labels when the button is pressed
        self.result_signal.emit("켜는중", 0)
        
class Ui_MainWindow(object):
    def setupUi(self, MainWindow):
        MainWindow.setObjectName("MainWindow")
        MainWindow.resize(640, 480)
        self.received_text = ""
        self.centralwidget = QtWidgets.QWidget(MainWindow)
        self.centralwidget.setObjectName("centralwidget")
        
        # Create a QScrollArea
        self.scrollArea = QtWidgets.QScrollArea(self.centralwidget)
        self.scrollArea.setGeometry(QtCore.QRect(40, 300, 561, 91))
        self.scrollArea.setWidgetResizable(True)
        self.scrollArea.setObjectName("scrollArea")

        # Create the label inside the QScrollArea
        self.label = QtWidgets.QLabel(self.scrollArea)
        self.label.setAlignment(QtCore.Qt.AlignTop)
        self.label.setObjectName("text_label")
        self.label.setWordWrap(True)
        self.scrollArea.setWidget(self.label)

        self.label_2 = QtWidgets.QLabel(self.centralwidget)
        self.label_2.setGeometry(QtCore.QRect(40, 30, 381, 211))
        self.label_2.setText("")
        self.label_2.setObjectName("imgae_image")

        self.pushButton = QtWidgets.QPushButton(self.centralwidget)
        self.pushButton.setGeometry(QtCore.QRect(470, 40, 141, 101))
        self.pushButton.setObjectName("pushButton")
        MainWindow.setCentralWidget(self.centralwidget)

        #english_text_label
        self.new_label = QtWidgets.QLabel(self.centralwidget)
        self.new_label.setGeometry(QtCore.QRect(40, 200, 561, 201))
        self.new_label.setText("")
        self.new_label.setAlignment(QtCore.Qt.AlignCenter)  # 텍스트 중앙 정렬
        self.new_label.setObjectName("english_text_label")
        self.new_label.hide()  # Hide the new label initially
        #english_image_label
        self.new_label_2 = QtWidgets.QLabel(self.centralwidget)
        self.new_label_2.setGeometry(QtCore.QRect(50, 40, 144, 130))
        self.new_label_2.setPixmap(QtGui.QPixmap("english_image.png"))
        self.new_label_2.setAlignment(QtCore.Qt.AlignCenter)
        self.new_label_2.setObjectName("english_image_label")
        self.new_label_2.hide()  # Hide the new label initially

        MainWindow.setCentralWidget(self.centralwidget)
        self.menubar = QtWidgets.QMenuBar(MainWindow)
        self.menubar.setGeometry(QtCore.QRect(0, 0, 800, 29))
        self.menubar.setObjectName("menubar")
        MainWindow.setMenuBar(self.menubar)
        self.statusbar = QtWidgets.QStatusBar(MainWindow)
        self.statusbar.setObjectName("statusbar")
        MainWindow.setStatusBar(self.statusbar)

        self.retranslateUi(MainWindow)
        QtCore.QMetaObject.connectSlotsByName(MainWindow)

        self.pushButton.clicked.connect(self.onPushButtonClicked)

        # Initialize SoundAnalysis and connect the result_signal with the updateLabel function
        self.sound_analysis = SoundAnalysis(model, device, transformation, SAMPLE_RATE)
        self.sound_analysis.result_signal.connect(self.updateLabel)
        self.sound_analysis.result_signal.connect(self.updateLabel2)
        self.sound_analysis.start()  # Start the sound analysis thread
        
        self.is_new_labels_visible = False

        

    def onPushButtonClicked(self): ######BUTTON#######
        global mode
        mode = not mode
        self.sound_analysis.reset_label()#reset the screen (its connected to the sound analysis class)
        print("mode now :  ",mode)

    def updateLabel(self, predicted_label, predicted_confidence):
        #print("Received signal")  # Print message when signal is received
        if predicted_confidence == 0:
            self.label.setText(f"{predicted_label}")
        else:
            self.label.setText(f"{predicted_label}  {predicted_confidence*100:.2f}%")
        #print(predicted_label)
        
        # After updating the text, ensure the QScrollArea scrolls to the bottom
        self.scrollArea.verticalScrollBar().setValue(self.scrollArea.verticalScrollBar().maximum())

        
        
    def updateLabel2(self, predicted_label):
        relative_image_folder_path = "../png" #\ only works for windows
        image_folder_path= os.path.join(current_path, relative_image_folder_path)
        full_file_name = os.path.join(image_folder_path, f"{predicted_label}.png")
        #self.movie = QMovie(full_file_name)
        #self.label.setMovie(self.movie)
        #self.movie.start()
        self.label_2.setPixmap(QtGui.QPixmap(full_file_name)) 
        #print(predicted_label)


    def retranslateUi(self, MainWindow):
        _translate = QtCore.QCoreApplication.translate
        MainWindow.setWindowTitle(_translate("MainWindow", "MainWindow"))
        self.label.setFont(QtGui.QFont("AppleSystemUIFont",20))
        self.label.setStyleSheet("Color : black")
        self.pushButton.setText(_translate("MainWindow", "PushButton"))
        
try:
    import RPi.GPIO as GPIO
    print("RPi.GPIO loaded successfully")
    import threading
    print("threading loaded successfully")

    class VibrationController:
        def __init__(self):
            # 핀 번호 설정
            self.red_button_pin = 17
            self.yellow_button_pin = 22
            self.green_button_pin = 27
            self.vibration_motor_pin = 18
            
            # 진동 세기 초기화
            self.vibration_intensity_temp = 100
            self.vibration_intensity = 0
            
            # 이전 스위치 상태 초기화
            self.prev_red_button_state = GPIO.HIGH
            self.prev_yellow_button_state = GPIO.HIGH
            self.prev_green_button_state = GPIO.HIGH
            
            self.debounce_time = 0.2
            
            # 진동 상태를 저장하는 변수
            self.vibration_on = False
            
            # GPIO 초기화
            GPIO.setmode(GPIO.BCM)
            GPIO.setup(self.red_button_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)
            GPIO.setup(self.yellow_button_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)
            GPIO.setup(self.green_button_pin, GPIO.IN, pull_up_down=GPIO.PUD_UP)
            GPIO.setup(self.vibration_motor_pin, GPIO.OUT)
            
            # PWM 설정
            self.pwm_frequency = 1000
            self.pwm = GPIO.PWM(self.vibration_motor_pin, self.pwm_frequency)
            self.pwm.start(self.vibration_intensity)
        
        def display_vibration_intensity(self):
            print(f"진동 세기: {self.vibration_intensity}")
        
        def adjust_vibration_intensity(self, button_pin):
            if button_pin == self.red_button_pin:
                self.vibration_intensity = self.vibration_intensity_temp
            elif button_pin == self.yellow_button_pin:
                self.vibration_intensity = max(self.vibration_intensity - 10, 0)
            elif button_pin == self.green_button_pin:
                self.vibration_intensity = min(self.vibration_intensity + 10, 100)
            
            self.pwm.ChangeDutyCycle(self.vibration_intensity)
            self.display_vibration_intensity()
        
        def run(self):
            try:
                while True:
                    red_button_state = GPIO.input(self.red_button_pin)
                    yellow_button_state = GPIO.input(self.yellow_button_pin)
                    green_button_state = GPIO.input(self.green_button_pin)
            
                    if red_button_state != self.prev_red_button_state:
                        time.sleep(self.debounce_time)
                        if red_button_state != GPIO.input(self.red_button_pin):
                            
                            if not self.vibration_on:
                                self.vibration_on = True
                                self.adjust_vibration_intensity(self.red_button_pin)
                            else:
                                self.vibration_intensity_temp = self.vibration_intensity
                                self.vibration_intensity = 0
                                self.pwm.ChangeDutyCycle(self.vibration_intensity)
                                self.display_vibration_intensity()
                                self.vibration_on = False
            
                        self.prev_red_button_state = red_button_state
            
                    if yellow_button_state != self.prev_yellow_button_state:
                        time.sleep(self.debounce_time)
                        if yellow_button_state != GPIO.input(self.yellow_button_pin):
                            self.adjust_vibration_intensity(self.yellow_button_pin)
                        self.prev_yellow_button_state = yellow_button_state
            
                    if green_button_state != self.prev_green_button_state:
                        time.sleep(self.debounce_time)
                        if green_button_state != GPIO.input(self.green_button_pin):
                            self.adjust_vibration_intensity(self.green_button_pin)
                        self.prev_green_button_state = green_button_state
            
                    time.sleep(0.01)  
            
            except KeyboardInterrupt:
                self.pwm.stop()
                GPIO.cleanup()
except:
    print("RPi.GPIO import failed. No vibration")

if __name__ == "__main__":
    import sys
    app = QtWidgets.QApplication(sys.argv)
    MainWindow = QtWidgets.QMainWindow()
    try:
        vibration_controller = VibrationController()
        vibration_thread = threading.Thread(target=vibration_controller.run)
        vibration_thread.start()
    except:
        print("pass vibration")
    ui = Ui_MainWindow()
    ui.setupUi(MainWindow)
    MainWindow.show()
    sys.exit(app.exec_())

 결과 값 : 
RPi.GPIO import failed. No vibration
pass vibration


qt.qpa.fonts: Populating font family aliases took 121 ms. Replace uses of missing font family "AppleSystemUIFont" with one that exists to avoid this cost. 


SystemExit: 0

/Users/owo/anaconda3/envs/torchenv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 가능한 폰트 확인

In [ ]:
from PyQt5.QtGui import QFontDatabase

print(QFontDatabase().families())